# Seq2Seq 인코더-디코더 구조

![img](E15im1.jpg)

Encoder = 데이터에서 Feature Extractor
Decoder = 데이터 재생성

![img](E15im2.png)

Seq2seq의 feature vector ? ==> 인코더의 입력 문장을 해석해서 만든 Hidden state vector

- A언어의 문장 X를 z라는 hidden state로 해석한 후 z를 다시 B 언어의 문장 Y로 재생성하는 것입니다. 그러므로 인코더에서 필요한 것은 인코더의 마지막 time step의 hidden state입니다. 그리고 이를 두번째 RNN인 디코더에 전달합니다.

- 디코더는 인코더의 마지막 time step의 hidden state를 전달 받아 자신의 초기 hidden state로 하고, 출력 문장을 생성해내기 시작합니다.

## Conditional Language Model
### 어떤 말을 만들고 싶은지 제어하기 위한 언어 모델 = 번역(seq2seq)
- 문장 X를 해석해서 c로 만드는 인코더를 또다른 RNN으로 만드는 것입니다. 그렇게 만든 c를 다시 문장생성기인 디코더 RNN에 입력으로 넣어주는 모델을 만들어 낸 것이 바로 오늘 다루게 될 seq2seq인 것입니다.

## Teacher Forcing(교사 강요)


![img](E15im3.png)

- seq2seq는 훈련 과정과 테스트 과정에서의 동작 방식이 다르다는 특징이 있습니다. 이전 스텝의 그림을 보면 디코더 RNN은 이전 time step의 출력을 현재 time step의 입력으로 사용한다는 특징을 가지고 있습니다. 그런데 이는 테스트 과정에서의 이야기이고, 훈련 과정은 조금 다른 방식을 사용합니다. 그 이유는 훈련 과정에서 이전 time step이 잘못된 예측을 한다면 이를 입력으로 한 현재 time step의 예측도 잘못될 수 있기 때문입니다. 이런 상황이 반복되면 훈련 시간이 굉장히 늘어나게 됩니다.

- 훈련 과정에서는 실제 정답 시퀀스를 알고 있는 상황이므로 이전 time step의 예측값을 현재 time step의 입력으로 사용하는 것이 아니라 이전 time step의 실제값으로 사용할 수 있습니다. 이 작업을 교사 강요(teacher forcing)이라고 합니다. 이 기법은 seq2seq 뿐 아니라 sequence 데이터의 생성모델에서 일반적으로 사용되는 기법이기도 합니다. 물론, 이는 모델이 훈련 데이터 외의 결과를 생성해내는 능력을 기르는데에 조금 방해가 될 수 있다는 단점도 존재합니다.

# 번역기 만들기

## 1. 데이터 전처리

```
$ mkdir -p ~/aiffel/translator_seq2seq/data
$ mkdir -p ~/aiffel/translator_seq2seq/models
$ wget https://www.manythings.org/anki/fra-eng.zip
$ mv fra-eng.zip  ~/aiffel/translator_seq2seq/data
$ cd ~/aiffel/translator_seq2seq/data && unzip fra-eng.zip
```


In [1]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np

In [2]:
import os
file_path = os.getenv('HOME')+'/aiffel/translator_seq2seq/data/fra.txt'
lines = pd.read_csv(file_path, names=['eng', 'fra', 'cc'], sep='\t')
print('전체 샘플의 수 :',len(lines))
lines.sample(5) #샘플 5개 출력

전체 샘플의 수 : 178009


,eng,fra,cc
165758,She got him to do everything she wanted him to...,Elle a obtenu qu'il fasse tout ce qu'elle voul...,CC-BY 2.0 (France) Attribution: tatoeba.org #8...
109075,Did you do yesterday's homework?,As-tu fait les devoirs d'hier ?,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
133797,Why did you become a police officer?,Pourquoi êtes-vous devenu agent de police ?,CC-BY 2.0 (France) Attribution: tatoeba.org #4...
24280,I hate these words.,Je déteste ces mots.,CC-BY 2.0 (France) Attribution: tatoeba.org #8...
103367,He likes geography and history.,Il aime la géographie et l'histoire.,CC-BY 2.0 (France) Attribution: tatoeba.org #3...


In [3]:
lines = lines[['eng', 'fra']][:50000] # 5만개 샘플 사용
lines.sample(5)

,eng,fra
2975,We all quit.,Nous arrêtons tous.
4904,Are they dead?,Sont-elles mortes ?
11306,I remember that.,Je me souviens de cela.
18493,Get on your knees.,Mets-toi à genoux.
12818,Tom is likeable.,Tom est sympathique.


In [4]:
# 시작 토큰과 종료 토큰 추가
sos_token = '\t'
eos_token = '\n'
lines.fra = lines.fra.apply(lambda x : '\t '+ x + ' \n')
print('전체 샘플의 수 :',len(lines))
lines.sample(5)

전체 샘플의 수 : 50000


,eng,fra
34893,He was fully clothed.,\t Il était entièrement vêtu. \n
3460,Have a donut.,\t Prends un beignet. \n
21524,Tom broke his arm.,\t Tom lui a cassé le bras. \n
16708,They've all left.,\t Elles sont toutes parties. \n
20754,She can jump high.,\t Elle peut sauter haut. \n


### 텍스틑 -> 정수 로 변환하는 인코딩

In [5]:
eng_tokenizer = Tokenizer(char_level=True)   # 문자 단위로 Tokenizer를 생성합니다. 
eng_tokenizer.fit_on_texts(lines.eng)               # 50000개의 행을 가진 eng의 각 행에 토큰화를 수행
input_text = eng_tokenizer.texts_to_sequences(lines.eng)    # 단어를 숫자값 인덱스로 변환하여 저장
input_text[:3]

[[19, 3, 8], [10, 5, 8], [10, 5, 8]]

In [6]:
fra_tokenizer = Tokenizer(char_level=True)   # 문자 단위로 Tokenizer를 생성합니다. 
fra_tokenizer.fit_on_texts(lines.fra)                 # 50000개의 행을 가진 fra의 각 행에 토큰화를 수행
target_text = fra_tokenizer.texts_to_sequences(lines.fra)     # 단어를 숫자값 인덱스로 변환하여 저장
target_text[:3]

[[11, 1, 19, 4, 1, 33, 1, 12],
 [11, 1, 3, 4, 13, 7, 5, 1, 33, 1, 12],
 [11, 1, 3, 4, 13, 7, 5, 14, 1, 12]]

#### 0번 토큰을 고려하여 size +1

In [7]:
eng_vocab_size = len(eng_tokenizer.word_index) + 1
fra_vocab_size = len(fra_tokenizer.word_index) + 1
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)

영어 단어장의 크기 : 51
프랑스어 단어장의 크기 : 73


In [10]:
max_eng_seq_len = max([len(line) for line in input_text])
max_fra_seq_len = max([len(line) for line in target_text])

In [11]:
print('전체 샘플의 수 :',len(lines))
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

전체 샘플의 수 : 50000
영어 단어장의 크기 : 51
프랑스어 단어장의 크기 : 73
영어 시퀀스의 최대 길이 23
프랑스어 시퀀스의 최대 길이 76


### 영어  : 인코더의 입력
### 프랑스어 : 1. 디코더의 출력과 비교해야할 정답 데이터         2. Teacher Forcing 을 위해 디코더의 입력으로 사용할 데이터


- 디코더의 입력으로 사용할 시퀀스는 < eos >토큰이 필요가 없고, 디코더의 출력과 비교할 시퀀스는 < sos >가 필요가 없기 때문입니다. 가령, 영어로 'I am a person'이라는 문장을 프랑스어 'Je suis une personne'로 번역하는 번역기를 만든다고 해봅시다. 훈련 과정에서 디코더는 '< sos > Je suis une personne'를 입력받아서 'Je suis une personne < eos >'를 예측하도록 훈련되므로, 이런 방식으로 생성된 두가지 버전의 시퀀스를 준비해야 합니다

In [13]:
encoder_input = input_text
# 종료 토큰 제거
decoder_input = [[ char for char in line if char != fra_tokenizer.word_index[eos_token] ] for line in target_text] 
# 시작 토큰 제거
decoder_target = [[ char for char in line if char != fra_tokenizer.word_index[sos_token] ] for line in target_text]

In [14]:
print(decoder_input[:3])
print(decoder_target[:3])

[[11, 1, 19, 4, 1, 33, 1], [11, 1, 3, 4, 13, 7, 5, 1, 33, 1], [11, 1, 3, 4, 13, 7, 5, 14, 1]]
[[1, 19, 4, 1, 33, 1, 12], [1, 3, 4, 13, 7, 5, 1, 33, 1, 12], [1, 3, 4, 13, 7, 5, 14, 1, 12]]


In [15]:
encoder_input = pad_sequences(encoder_input, maxlen = max_eng_seq_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen = max_fra_seq_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen = max_fra_seq_len, padding='post')
print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 데이터의 크기(shape) : (50000, 23)
프랑스어 입력데이터의 크기(shape) : (50000, 76)
프랑스어 출력데이터의 크기(shape) : (50000, 76)


In [16]:
print(encoder_input[0])

[19  3  8  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]


### one-hot Encoding

In [17]:
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)
print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 데이터의 크기(shape) : (50000, 23, 51)
프랑스어 입력데이터의 크기(shape) : (50000, 76, 73)
프랑스어 출력데이터의 크기(shape) : (50000, 76, 73)


#### 5만건 중, 3만건만 검증 데이터로 

In [18]:
n_of_val = 3000

encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

print('영어 학습데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 학습 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 학습 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 학습데이터의 크기(shape) : (50000, 23, 51)
프랑스어 학습 입력데이터의 크기(shape) : (50000, 76, 73)
프랑스어 학습 출력데이터의 크기(shape) : (50000, 76, 73)


In [19]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model


⏳


### Encoder 설계하기
 1. LSTM 셀 설계
 2. 문장 입력
 3. 마지막 Time step의 Hideen state 와 cell state를 저장
 

- encoder의 hidden state는 디코더의 첫번째 hidden state
 - LSTM이라 cell state 까지 추가

In [20]:
# 입력 텐서 생성.
encoder_inputs = Input(shape=(None, eng_vocab_size))
# hidden size가 256인 인코더의 LSTM 셀 생성
encoder_lstm = LSTM(units = 256, return_state = True)
# 디코더로 전달할 hidden state, cell state를 리턴. encoder_outputs은 여기서는 불필요.
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
# hidden state와 cell state를 다음 time step으로 전달하기 위해서 별도 저장.
encoder_states = [state_h, state_c]

### Code 분석
1. 우선 LSTM의 입력 텐서를 정의해줍니다. 입력 문장을 저장하게 될 변수 텐서입니다.

2. 256의 hidden_size를 가지는 LSTM 셀을 만들어줍니다. LSTM의 수용력(capacity)를 의미합니다. return_state = True를 해서 hidden state와 cell state를 리턴받을 수 있도록 합니다.

3. 입력 텐서를 입력으로 마지막 time step의 hidden state와 cell state를 결과로 받습니다.

4. 마지막 time step의 hidden state와 cell state를 encoder_states라는 하나의 변수에 저장해뒀습니다. 이를 디코더에 전달하면 되겠네요.


### Decoer 설계하기

In [21]:
# 입력 텐서 생성.
decoder_inputs = Input(shape=(None, fra_vocab_size))
# hidden size가 256인 인코더의 LSTM 셀 생성
decoder_lstm = LSTM(units = 256, return_sequences = True, return_state=True)
# decoder_outputs는 모든 time step의 hidden state
decoder_outputs, _, _= decoder_lstm(decoder_inputs, initial_state = encoder_states)

- 세번째 줄을 보면 디코더의 인자로 initial_state : LSTM 셀의 초기 상태를 정의해주는 인자. (여기서는 Encoder의 마지막 time step의 hidden , cell state

In [22]:
decoder_softmax_layer = Dense(fra_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

In [23]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 51)]   0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None, 73)]   0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 256), (None, 315392      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, None, 256),  337920      input_2[0][0]                    
                                                                 lstm[0][1]            

In [24]:
model.fit(x=[encoder_input_train, decoder_input_train], y=decoder_target_train, \
          validation_data = ([encoder_input_test, decoder_input_test], decoder_target_test),
          batch_size=128, epochs=50)

Epoch 1/50
368/368 [==============================] - 6s 17ms/step - loss: 0.9401 - val_loss: 0.8105
Epoch 2/50
368/368 [==============================] - 6s 16ms/step - loss: 0.5720 - val_loss: 0.6464
Epoch 3/50
368/368 [==============================] - 6s 15ms/step - loss: 0.4670 - val_loss: 0.5683
Epoch 4/50
368/368 [==============================] - 6s 15ms/step - loss: 0.4101 - val_loss: 0.5086
Epoch 5/50
368/368 [==============================] - 6s 15ms/step - loss: 0.3722 - val_loss: 0.4792
Epoch 6/50
368/368 [==============================] - 6s 15ms/step - loss: 0.3448 - val_loss: 0.4518
Epoch 7/50
368/368 [==============================] - 6s 16ms/step - loss: 0.3236 - val_loss: 0.4390
Epoch 8/50
368/368 [==============================] - 6s 16ms/step - loss: 0.3071 - val_loss: 0.4238
Epoch 9/50
368/368 [==============================] - 6s 16ms/step - loss: 0.2934 - val_loss: 0.4055
Epoch 10/50
368/368 [==============================] - 6s 16ms/step - loss: 0.2822 - val_lo

## 모델 테스트 하기
1. 인코더에 입력 문장을 넣어 마지막 time step의 hidden, cell state를 얻는다.
2. 토큰인 '\t'를 디코더에 입력한다.
3. 이전 time step의 출력층의 예측 결과를 현재 time step의 입력으로 한다.
4. 3을 반복하다가 토큰인 '\n'가 예측되면 이를 중단한다

#### Encoder 정의하기

In [25]:
encoder_model = Model(inputs = encoder_inputs, outputs = encoder_states)
encoder_model.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, 51)]        0         
_________________________________________________________________
lstm (LSTM)                  [(None, 256), (None, 256) 315392    
Total params: 315,392
Trainable params: 315,392
Non-trainable params: 0
_________________________________________________________________


### Decoder 정의하기

In [26]:
# 이전 time step의 hidden state를 저장하는 텐서
decoder_state_input_h = Input(shape=(256,))
# 이전 time step의 cell state를 저장하는 텐서
decoder_state_input_c = Input(shape=(256,))
# 이전 time step의 hidden state와 cell state를 하나의 변수에 저장
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# decoder_states_inputs를 현재 time step의 초기 상태로 사용.
# 구체적인 동작 자체는 def decode_sequence()에 구현.
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state = decoder_states_inputs)
# 현재 time step의 hidden state와 cell state를 하나의 변수에 저장.
decoder_states = [state_h, state_c]

#### Decoder 출력층 재설계

In [27]:
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)
decoder_model.summary()

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None, 73)]   0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 256)]        0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 256)]        0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, None, 256),  337920      input_2[0][0]                    
                                                                 input_3[0][0]         

#### Dictionary 생성 ( 단어 -> 정수 , 정수-> 단어)

In [28]:
eng2idx = eng_tokenizer.word_index
fra2idx = fra_tokenizer.word_index
idx2eng = eng_tokenizer.index_word
idx2fra = fra_tokenizer.index_word

- decode_sequence()의 입력으로 들어가는 것은 번역하고자 하는 문장의 정수 시퀀스입니다. decode_sequence() 내부에는 인코더를 구현한 encoder_model이 있어서 이 모델에 번역하고자 하는 문장의 정수 시퀀스인 'input_seq'를 입력하면, encoder_model은 마지막 시점의 hidden state를 리턴합니다.
- hidden state는 디코더의 첫번째 시점의 hidden state가 되고, 디코더는 이제 번역 문장을 완성하기 위한 예측 과정을 진행합니다. 디코더의 예측 과정에서는 이전 시점에서 예측한 단어를 디코더의 현재 시점의 입력으로 넣어주는 작업을 진행합니다. 그리고 이 작업은 종료를 의미하는 종료 토큰을 만나거나, 주어진 최대 길이를 넘을 때까지 반복

In [29]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # <SOS>에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1, 1, fra_vocab_size))
    target_seq[0, 0, fra2idx['\t']] = 1.

    stop_condition = False
    decoded_sentence = ""

    # stop_condition이 True가 될 때까지 루프 반복
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 문자로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = idx2fra[sampled_token_index]

        # 현재 시점의 예측 문자를 예측 문장에 추가
        decoded_sentence += sampled_char

        # <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_fra_seq_len):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1, 1, fra_vocab_size))
        target_seq[0, 0, sampled_token_index] = 1.

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

In [30]:
import numpy as np
for seq_index in [3,50,100,300,1001]: # 입력 문장의 인덱스 (자유롭게 선택해 보세요)
    input_seq = encoder_input[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print(35 * "-")
    print('입력 문장:', lines.eng[seq_index])
    print('정답 문장:', lines.fra[seq_index][1:len(lines.fra[seq_index])-1]) # '\t'와 '\n'을 빼고 출력
    print('번역기가 번역한 문장:', decoded_sentence[:len(decoded_sentence)-1]) # '\n'을 빼고 출력

-----------------------------------
입력 문장: Run!
정답 문장:  Cours ! 
번역기가 번역한 문장:  cours ! 
-----------------------------------
입력 문장: I left.
정답 문장:  Je suis partie. 
번역기가 번역한 문장:  je suis tombée. 
-----------------------------------
입력 문장: Call us.
정답 문장:  Appelez-nous ! 
번역기가 번역한 문장:  appelez-vous ! 
-----------------------------------
입력 문장: How nice!
정답 문장:  Comme c'est gentil ! 
번역기가 번역한 문장:  comme c'est blanc ! 
-----------------------------------
입력 문장: Turn left.
정답 문장:  Tourne à gauche. 
번역기가 번역한 문장:  tourne à gauche. 


 데이터에서 상위 33,000개의 샘플만 사용해주세요
 
 

## 1. 정제, 정규화, 전처리 (영어, 프랑스어)

### 1-1. 구두점(Punctuation)을 단어와 분리

- 분리 전 : he is a Good boy!

- 분리 후 : he is a Good boy !

### 2. 소문자로 변환

- 변환 전 : he is a Good boy !

- 변환 후 : he is a good boy !

### 3. 띄어쓰기 단위로 토큰를 수행하세요.

- 토큰화 전 : 'he is a good boy !'

- 토큰화 후 : ['he', 'is', 'a', 'good', 'boy', '!']


## 2. 디코더의 문장에 시작 토큰과 종료 토큰을 넣어기

- 입력 시퀀스 : ['', 'courez', '!']

- 레이블 시퀀스 : ['courez', '!', ']

## 3. 케라스의 토크나이저로 텍스트를 숫자로 바꾸기

 - 아래 링크의 2. 케라스 텍스트 전처리 참고
https://wikidocs.net/31766

- 영어와 프랑스어에 대한 토크나이저를 각각 생성하고, tokenizer.texts_to_sequences()를 사용하여 모든 샘플에 대해서 정수 시퀀스로 변환

## 4. 임베딩 층(Embedding layer) 사용하기
 - 입력이 되는 각 단어를 임베딩 층을 사용해서 벡터화 하기. 아래 링크의 1. 케라스 임베딩층 참고

 - https://wikidocs.net/33793 
 
 - 코드 예시)

```
from tensorflow.keras.layers import Input, mbedding, Masking

# 인코더에서 사용할 임베딩 층 사용 예시
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(단어장의 크기, 임베딩 벡터의 차원)(encoder_inputs)
encoder_lstm = LSTM(hidden state의 크기, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
```

#### 주의할 점은 인코더와 디코더의 임베딩 층은 서로 다른 임베딩 층을 사용해야 하지만,

디코더의 훈련 과정과 테스트 과정(예측 과정)에서의 임베딩 층은 동일해야 합니다!

## 5. 모델 구현하기

## 6. 모델 평가하기